In [1]:
import numpy as np

WIDTH = 395
HEIGHT = 860

train_data = np.load('E:/project data/training_data.npy',allow_pickle=True)


In [2]:

import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [3]:
import cv2
X=np.zeros([13000,150,300,3], dtype=np.uint16)
test_x=np.zeros([4000,150,300,3], dtype=np.uint16)
for i in range(len(train_data)):
    if i < 13000:
        a=train_data[i,0][80:475,50:910]
        a=cv2.cvtColor(a,cv2.COLOR_GRAY2RGB)
        a=cv2.resize(a,(300,150))
        X[i]=a
    else:
        a=train_data[i,0][80:475,50:910]
        a=cv2.cvtColor(a,cv2.COLOR_GRAY2RGB)
        a=cv2.resize(a,(300,150))
        test_x[i-13000]=a

In [4]:
import pandas as pd

val=pd.read_csv('C:/Users/Ant Pc/GitHub/AI-in-Mfg---Automated-Driving/Aaftab/choice.csv')
print(val.head())
Y=val.iloc[0:13000,0]
test_y=val.iloc[13000:,0]

   0
0  0
1  0
2  0
3  0
4  0


In [5]:
print(X.shape)
print(test_x.shape)
print(Y.shape)
print(test_y.shape)

(13000, 150, 300, 3)
(4000, 150, 300, 3)
(13000,)
(4000,)


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications
input_shape = (150, 300,3)
nclass = 10

base_model = applications.InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(150, 300,3))
base_model.trainable = True

add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dropout(0.5))

add_model.add(Dense(4096*2, 
                    activation='relu'))

add_model.add(Dense(4096*2, 
                    activation='relu'))

add_model.add(Dense(4096*2, 
                    activation='relu'))

add_model.add(Dense(4096, 
                    activation='relu'))
add_model.add(Dense(nclass, 
                    activation='softmax'))

model = add_model

model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.SGD(lr=1e-3, 
                                       momentum=0.9),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 3, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 8192)              16785408  
_________________________________________________________________
dense_1 (Dense)              (None, 8192)              67117056  
_________________________________________________________________
dense_2 (Dense)              (None, 8192)              67117056  
_________________________________________________________________
dense_3 (Dense)              (None, 4096)              3

In [7]:
#model.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])
#model = keras.models.load_model('C:/Users/Ant Pc/GitHub/AI-in-Mfg---Automated-Driving/Aaftab/keras.h5')

In [8]:
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 10 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 10
print(Y.shape)
Y=to_categorical(Y)
test_y=to_categorical(test_y)

X=X.reshape([13000,150,300,3])
test_x=test_x.reshape([4000,150,300,3])




(13000,)


In [9]:
#Verifying the dimension after one hot encoding
print(X.shape)
print(test_x.shape)
print(Y.shape)
print(test_y.shape)

(13000, 150, 300, 3)
(4000, 150, 300, 3)
(13000, 10)
(4000, 10)


In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )

val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)


In [11]:
train_generator.fit(X)
val_generator.fit(test_x)

In [12]:
batch_size= 2
epochs=100
learn_rate=.001

In [13]:
cw={0: 0.5057578586990352,
 2: 1.184942120134901,
 3: 2.3679417122040074,
 4: 722.2222222222222,
 5: 3.761574074074074,
 6: 0.19086210946676063,
 7: 8.253968253968255,
 8: 9.027777777777779,
 9: 55.55555555555556}

In [14]:
h=model.fit(train_generator.flow(X,Y, batch_size=batch_size), epochs = epochs, steps_per_epoch = X.shape[0]//batch_size, validation_data = val_generator.flow(test_x, test_y, batch_size=batch_size),class_weight=cw,verbose=1)


Epoch 1/100
6500/6500 [==============================] - 351s 54ms/step - loss: 2.7794 - accuracy: 0.0851 - val_loss: 2.0116 - val_accuracy: 0.0742
Epoch 2/100
6500/6500 [==============================] - 351s 54ms/step - loss: nan - accuracy: 0.1970 - val_loss: nan - val_accuracy: 0.2072
Epoch 3/100
1639/6500 [======>.......................] - ETA: 4:01 - loss: nan - accuracy: 0.2163

KeyboardInterrupt: 

In [ ]:
h.history

In [ ]:
model.save('keras.h5')

In [ ]:
import numpy as np
from grabscreen import grab_screen
import cv2
import time
from directkeys import PressKey,ReleaseKey, W, A, S, D
from getkeys import key_check

import random
t_time = 0.09

def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    ReleaseKey(S)

def left():
    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    PressKey(A)
    ReleaseKey(S)
    ReleaseKey(D)
    #ReleaseKey(S)

def right():
    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(S)
    
def reverse():
    PressKey(S)
    ReleaseKey(A)
    ReleaseKey(W)
    ReleaseKey(D)


def forward_left():
    PressKey(W)
    PressKey(A)
    ReleaseKey(D)
    ReleaseKey(S)
    
    
def forward_right():
    PressKey(W)
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(S)

    
def reverse_left():
    PressKey(S)
    PressKey(A)
    ReleaseKey(W)
    ReleaseKey(D)

    
def reverse_right():
    PressKey(S)
    PressKey(D)
    ReleaseKey(W)
    ReleaseKey(A)
    
def no_keys():

    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    ReleaseKey(A)
    ReleaseKey(S)
    ReleaseKey(D)

def main():
    last_time = time.time()
    for i in list(range(4))[::-1]:
        print(i+1)
        time.sleep(1)

    paused = False
    while(True):
        
        if not paused:
            # 800x600 windowed mode
            #screen =  np.array(ImageGrab.grab(bbox=(0,40,800,640)))
            screen = grab_screen(region=(0,40,3840,2160))
            last_time = time.time()
            screen = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)
            screen = cv2.resize(screen, (0,0), fx=0.25, fy=0.25)
            screen=cv2.cvtColor(screen,cv2.COLOR_GRAY2RGB)
            screen=screen[80:475,50:910]
            screen = cv2.resize(screen, (150,300))

            prediction = model.predict([screen.reshape(1,150,300,3)])[0]
            

            mode_choice = np.argmax(prediction)
            print(prediction)
            if mode_choice == 6:
                straight()
                choice_picked = 'straight'
                
            elif mode_choice == 8:
                reverse()
                choice_picked = 'reverse'
                
            elif mode_choice == 7:
                left()
                choice_picked = 'left'
            elif mode_choice == 9:
                right()
                choice_picked = 'right'
            elif mode_choice == 0:
                forward_left()
                choice_picked = 'forward+left'
            elif mode_choice == 2:
                forward_right()
                choice_picked = 'forward+right'
            elif mode_choice == 3:
                reverse_left()
                choice_picked = 'reverse+left'
            elif mode_choice == 5:
                reverse_right()
                choice_picked = 'reverse+right'
            else:
                no_keys()
                print('no')
                choice_picked = 'nokeys'

        keys = key_check()
        print(choice_picked)
        # p pauses game and can get annoying.
        if 'T' in keys:
            if paused:
                paused = False
                time.sleep(1)
            else:
                paused = True
                ReleaseKey(A)
                ReleaseKey(W)
                ReleaseKey(D)
                time.sleep(1)



In [ ]:
main()